In [5]:
from utils import print_score
from preprocess import preprocess_fraud_data

from sklearn.ensemble import RandomForestClassifier

import pandas as pd 
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [6]:
ruta = "./data/dataset_balanceado.csv"
df = pd.read_csv(ruta)

data = preprocess_fraud_data(df)

X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']

**Modelo Base de Random Forest** 

In [7]:
modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

print_score(modelo, X_train, y_train, X_val, y_val, train=True)
print_score(modelo, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 99.98%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.999833    1.000000  0.999848     0.999917      0.999849
recall        1.000000    0.998333  0.999848     0.999167      0.999848
f1-score      0.999917    0.999166  0.999848     0.999541      0.999848
support    6000.000000  600.000000  0.999848  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[6000    0]
 [   1  599]]

Validation Result:
Accuracy Score: 90.86%
_______________________________________________
CLASSIFICATION REPORT:
                     0      1  accuracy    macro avg  weighted avg
precision     0.909050    0.0  0.908636     0.454525      0.826409
recall        0.999500    0.0  0.908636     0.499750      0.908636
f1-score      0.952131    0.0  0.908636     0.476066      0.865574
support    2000.000000  200.0  0.908636  2200.000000   

> El modelo base Random Forest con sus parámetros por defecto, sufre de `sobreajuste` (overfitting), lo que significa que ha vuelto un experto en memorizar los datos de entrenamiento, pero no es capaz de detectar ningún fraude en datos que nunca ha visto, lo cual lo convierte en un modelo inútil.

In [9]:
# 'modelo.estimators_' es una lista de todos los árboles de decisión entrenados
profundidades = [arbol.get_depth() for arbol in modelo.estimators_]

print(f"El modelo tiene {len(profundidades)} árboles.")
print("-" * 30)
print(f"Profundidad máxima encontrada: {max(profundidades)}")
print(f"Profundidad mínima encontrada: {min(profundidades)}")
print(f"Profundidad promedio: {np.mean(profundidades):.2f}")

print("\nProfundidades de los primeros 10 árboles:")
print(profundidades[:10])

El modelo tiene 100 árboles.
------------------------------
Profundidad máxima encontrada: 33
Profundidad mínima encontrada: 21
Profundidad promedio: 25.85

Profundidades de los primeros 10 árboles:
[22, 23, 22, 22, 24, 25, 23, 23, 27, 24]


> El modelo cuenta con 100 árboles, esto árboles tienen una profundidad máxima de 33, la mínima de 21 y el promedio es de casi 26 niveles.
>
> Al no ponerle límites (max_depth=None) a los árboles, se permitió que cada árbol creciera sin control, lo que los llevó a seguir bajando y hacer preguntas muy específicas permitiendoles ser demasiados detallados, lo cual les permite aprende hasta el mínimo detalle y memorizarlo, por eso es que el modelo funciona tan bien en entrenamiento pero es pésimo en validación, porque son datos que nunca a visto, el modelo solo memoriza no sabe generalizar.

**Podar árboles:** Limitar la profundidad y que cada hoja tenga un número mínimo de ejemplos.

In [15]:
modelo_mejorado = RandomForestClassifier(max_depth=10, min_samples_leaf=10, min_samples_split=10, random_state=42)
modelo_mejorado.fit(X_train, y_train)

print_score(modelo_mejorado, X_train, y_train, X_val, y_val, train=True)
print_score(modelo_mejorado, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 90.91%
_______________________________________________
CLASSIFICATION REPORT:
                     0      1  accuracy    macro avg  weighted avg
precision     0.909091    0.0  0.909091     0.454545      0.826446
recall        1.000000    0.0  0.909091     0.500000      0.909091
f1-score      0.952381    0.0  0.909091     0.476190      0.865801
support    6000.000000  600.0  0.909091  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[6000    0]
 [ 600    0]]

Validation Result:
Accuracy Score: 90.91%
_______________________________________________
CLASSIFICATION REPORT:
                     0      1  accuracy    macro avg  weighted avg
precision     0.909091    0.0  0.909091     0.454545      0.826446
recall        1.000000    0.0  0.909091     0.500000      0.909091
f1-score      0.952381    0.0  0.909091     0.476190      0.865801
support    2000.000000  200.0  0.909091  2200.000000   2200.000000
_____________

> Al podar los árboles con max_depth, min_samples_leaf y min_samples_split, se ha resuelto el problema del sobreajuste, sin embargo, al hacerlo, se ha vuelto al problema inicial de nuestros datos, el desbalance de clases. El modelo actual, aunque ya no memoriza, sigue siendo incapaz de identificar el fraude, lo que lo mantiene siendo un modelo inútil.
>
> Ahora que el modelo no puede memorizar casos específicos (debido a la poda), se ve forzado a aprender reglas generales. Dado que la clase fraude, es la clase minoritaria, la estrategia por la que opta el modelo es ignorar por completo a la clase con menor cantidda e muestras, el algoritmo concluye que el costo de equivocarse al intentar predecir un fraude es mayor que el beneficio de acertar, por lo que adopta la política de siempre predecir la clase mayoritaria. 

**class_weight='balanced'**

Ajuste de pesos de clase., esta técnica funcionará como un multiplicador de penalización.      

In [16]:
modelo_balanceado = RandomForestClassifier(max_depth=10, min_samples_leaf=10, min_samples_split=10, class_weight='balanced', random_state=42)
modelo_balanceado.fit(X_train, y_train)

print_score(modelo_balanceado, X_train, y_train, X_val, y_val, train=True)
print_score(modelo_balanceado, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 94.68%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.970672    0.707846  0.946818     0.839259      0.946778
recall        0.970833    0.706667  0.946818     0.838750      0.946818
f1-score      0.970752    0.707256  0.946818     0.839004      0.946798
support    6000.000000  600.000000  0.946818  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[5825  175]
 [ 176  424]]

Validation Result:
Accuracy Score: 86.32%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.908217    0.075630  0.863182     0.491924      0.832527
recall        0.945000    0.045000  0.863182     0.495000      0.863182
f1-score      0.926244    0.056426  0.863182     0.491335      0.847169
support    2000.000000  200.000000 

> Al combinar la regularización para controlar el sobreajuste con el parámetro class_weight='balanced' para atacar el desbalanceo, el modelo ha aprendido a identificar transacciones fraudulentas, se ha pasado de un modelo que ignoraba por completo el fraude (Recall de 0.0) a uno que lo detecta, sin embargo, los resultados actuales generan muchas falsas alarmas, y todavía no es capaz de identificar correctamente todos los verdaderos positivos.

**SMOTE**

In [17]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Pipeline que primero aplica SMOTE y luego entrena el clasificador regularizado
pipeline_smote = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(max_depth=10, min_samples_leaf=10, random_state=42))
])
pipeline_smote.fit(X_train, y_train)

print_score(pipeline_smote, X_train, y_train, X_val, y_val, train=True)
print_score(pipeline_smote, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 84.56%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.927114    0.227568  0.845606     0.577341      0.863519
recall        0.901000    0.291667  0.845606     0.596333      0.845606
f1-score      0.913870    0.255661  0.845606     0.584766      0.854033
support    6000.000000  600.000000  0.845606  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[5406  594]
 [ 425  175]]

Validation Result:
Accuracy Score: 81.41%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.907322    0.076923  0.814091     0.492123      0.831831
recall        0.886000    0.095000  0.814091     0.490500      0.814091
f1-score      0.896534    0.085011  0.814091     0.490773      0.822759
support    2000.000000  200.000000 

##### SMOTE

Balancear el set de entrenamiento creando nuevos ejemplos sintéticos de fraude lo cual le dará al modelo muchos más ejemplos de los que aprender.

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

modelo_final = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=5, min_samples_split=10, random_state=42)

modelo_final.fit(X_train_resampled, y_train_resampled)

print_score(modelo_final, X_train_resampled, y_train_resampled, X_val, y_val, train=True)
print_score(modelo_final, X_train_resampled, y_train_resampled, X_val, y_val, train=False)

Train Result:
Accuracy Score: 85.51%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy      macro avg   weighted avg
precision      0.846829      0.863671  0.855051       0.855250       0.855250
recall         0.866902      0.843199  0.855051       0.855051       0.855051
f1-score       0.856748      0.853312  0.855051       0.855030       0.855030
support    59400.000000  59400.000000  0.855051  118800.000000  118800.000000
_______________________________________________
Confusion Matrix: 
 [[51494  7906]
 [ 9314 50086]]


--- Resultados del Modelo con SMOTE en el set de Validación ---
Test Result:
Accuracy Score: 85.95%
_______________________________________________
CLASSIFICATION REPORT:
                      0           1  accuracy     macro avg  weighted avg
precision      0.989964    0.009763   0.85945      0.499864      0.980162
recall         0.866818    0.130000   0.85945      0.498409      0.859450
f1-score

##### Ajustar hiperpárametros

In [ ]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(n_estimators=100, random_state=42)

param_grid = {
    'max_depth': [10, 20, None],          # Profundidad del árbol. None = sin límite.
    'min_samples_leaf': [2, 5, 10],       # Mínimo de muestras en una hoja.
    'min_samples_split': [5, 10, 20],     # Mínimo para dividir un nodo.
    'criterion': ['gini', 'entropy']      # Criterio para medir la calidad de una división.
}

# métrica de evaluación
scoring_metric =  'f1_macro'

# Configurar GridSearchCV
# cv=3 significa 3-fold cross-validation. Para cada una de las 54 combinaciones,
# entrenará y validará el modelo 3 veces. Total de entrenamientos: 54 * 3 = 162.
grid_search = GridSearchCV(estimator=rf, 
                            param_grid=param_grid, 
                            cv=3, 
                            scoring=scoring_metric,
                            verbose=2) # verbose=2 te mostrará el progreso

print(f"\nIniciando GridSearchCV... Probando {len(param_grid['max_depth'])*len(param_grid['min_samples_leaf'])*len(param_grid['min_samples_split'])*len(param_grid['criterion'])} combinaciones.")
grid_search.fit(X_train_resampled, y_train_resampled)

print("\n--- Resultados de GridSearchCV ---")
print(f"Mejor puntuación ({scoring_metric}): {grid_search.best_score_:.4f}")
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

best_rf_model = grid_search.best_estimator_

print("\n--- Resultados del Modelo OPTIMIZADO en el set de Validación ---")
print_score(best_rf_model, X_train_resampled, y_train_resampled, X_val, y_val, train=False)



Iniciando GridSearchCV... Probando 54 combinaciones.
Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5; total time=  11.6s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5; total time=   9.3s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5; total time=  11.5s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=10; total time=  11.6s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=10; total time=   9.9s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=10; total time=  10.2s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=20; total time=  10.6s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=20; total time=  12.0s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=20; total 